Function to use methods from FraudKagglePreProcessData module to process train or test data.  Saves result as a df and as a pickled file for easy loading later.

In [1]:
from FraudKagglePreProcessData import PreProcessData
from datetime import datetime
from datetime import datetime as dt
import pandas as pd
import numpy as np
import re

In [2]:
def load_and_merge_data(transaction_csv,identity_csv,isTrain):
    df_transaction = pd.read_csv(transaction_csv, index_col='TransactionID')
    df_identity = pd.read_csv(identity_csv, index_col='TransactionID')
    df = pd.merge(df_transaction, df_identity, on='TransactionID', how='left')
    del df_transaction
    del df_identity
    if isTrain:
        labels = df[['isFraud']]
        df.pop('isFraud')
    else:
        labels = []
    return df, labels

In [3]:
train,labels  = load_and_merge_data('./data/raw/train_transaction_small.csv','./data/raw/train_identity_small.csv',isTrain=True)
test,testlabels  = load_and_merge_data('./data/raw/test_transaction_small.csv','./data/raw/test_identity_small.csv',isTrain=False)

In [4]:
def get_lists_of_numerical_categorical(df,regex):
    #Regex for categorical fields:
    categorical = []
    numerical = []

    #Create lists of categorical and numeircal fields:
    for i in df:
        if re.match(regex, i):
            categorical.append(i)
        else:
            numerical.append(i)
    return numerical,categorical
cat_columns_regex='ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38'
numerical,categorical = get_lists_of_numerical_categorical(train,cat_columns_regex)

In [47]:
# start=datetime.now()
# print(start)
# train,labels  = load_and_merge_data('./data/raw/train_transaction_small.csv','./data/raw/train_identity_small.csv',isTrain=True)
# #train,NAs = PreProcessData.reduce_mem_usage(train)
# #train = PreProcessData.process_dates(train)
# test,testlabels  = load_and_merge_data('./data/raw/test_transaction_small.csv','./data/raw/test_identity_small.csv',isTrain=False)
# #test,NAs = PreProcessData.reduce_mem_usage(test)
# #test = PreProcessData.process_dates(test)
# finish=datetime.now()
# print('time taken: '+str(finish-start))
# features=list(train.columns)

2019-09-13 10:41:45.620488


NameError: name 'numerically_encode_string_categoricals' is not defined

In [5]:
def get_lists_of_numerical_categorical(df,regex):
    #Regex for categorical fields:
    categorical = []
    numerical = []

    #Create lists of categorical and numeircal fields:
    for i in df:
        if re.match(regex, i):
            categorical.append(i)
        else:
            numerical.append(i)
    return numerical,categorical
cat_columns_regex='ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38'
numerical,categorical = get_lists_of_numerical_categorical(train,cat_columns_regex)

In [6]:
train.isna().sum().sum()

97024

In [45]:
#def impute_NAs(df,categorical):
#    df[categorical] = df[categorical].apply(lambda x: x.fillna(x.value_counts().index[0]))
#    return df
train[categorical] = train[categorical].apply(lambda x: x.fillna(x.value_counts().index[0]))

train[categorical].isna().sum()
#train = impute_NAs(train,categorical)

ProductCD        0
card1            0
card2            0
card3            0
card4            0
card5            0
card6            0
addr1            0
addr2            0
P_emaildomain    0
R_emaildomain    0
M1               0
M2               0
M3               0
M4               0
M5               0
M6               0
M7               0
M8               0
M9               0
id_12            0
id_13            0
id_14            0
id_15            0
id_16            0
id_17            0
id_18            0
id_19            0
id_20            0
id_21            0
id_22            0
id_23            0
id_24            0
id_25            0
id_26            0
id_27            0
id_28            0
id_29            0
id_30            0
id_31            0
id_32            0
id_33            0
id_34            0
id_35            0
id_36            0
id_37            0
id_38            0
DeviceType       0
DeviceInfo       0
dtype: int64

In [23]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
train = reduce_mem_usage(train)

Memory usage of dataframe is 1.65 MB
Memory usage after optimization is: 0.40 MB
Decreased by 75.5%
Wall time: 340 ms


In [21]:
train.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,86400,68.5,4,13926,NaN,150.0,1,142.0,1,315.0,...,103,NaN,260,4,2,2,2,2,2,1742
2987001,86401,29.0,4,2755,404.0,150.0,2,102.0,1,325.0,...,103,NaN,260,4,2,2,2,2,2,1742
2987002,86469,59.0,4,4663,490.0,150.0,4,166.0,2,330.0,...,103,NaN,260,4,2,2,2,2,2,1742
2987003,86499,50.0,4,18132,567.0,150.0,2,117.0,2,476.0,...,103,NaN,260,4,2,2,2,2,2,1742
2987004,86506,50.0,1,4497,514.0,150.0,2,102.0,1,420.0,...,124,32.0,164,3,1,0,1,1,1,954


In [29]:
#train.to_pickle('./data/interim/train_joined.pkl')
#test.to_pickle('./data/interim/test_joined.pkl')
#labels.to_pickle('./data/interim/labels.pkl')

In [9]:
train = pd.read_pickle('./data/interim/train_joined.pkl')
test = pd.read_pickle('./data/interim/test_joined.pkl')
labels = pd.read_pickle('./data/interim/labels.pkl')

In [26]:
features=list(train.columns)

In [7]:
test = PreProcessData.process_dates(test)
train = PreProcessData.process_dates(train)

In [ ]:
train.isna().sum()

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn import metrics
start=datetime.now()
%time
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2,random_state=42)
finish=datetime.now()

Wall time: 0 ns


In [31]:
start=datetime.now()
print(start)
rf = RandomForestClassifier(random_state=42,n_estimators = 50, max_depth=25)
rf.fit(X_train, y_train)
finish=datetime.now()
print('time taken: '+str(finish-start))
important_features = pd.DataFrame(rf.feature_importances_,features).sort_values(0,ascending=False).head(10)
important_features

2019-09-12 16:45:00.813828
time taken: 0:01:44.486402


,0
C1,0.026922
C13,0.020372
card2,0.015228
C11,0.014771
TransactionAmt,0.014195
card1,0.013934
C14,0.013922
C6,0.013905
C2,0.012952
C4,0.011906


In [24]:
X_train['time_hour'].dtype

dtype('int64')

In [25]:
y_pred_prob=rf.predict_proba(X_test)
y_pred=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("ROC area under:",metrics.roc_auc_score(y_test, y_pred_prob[:,1]))

Accuracy: 0.9789006671859654
ROC area under: 0.9401913873171168


In [22]:
from joblib import dump, load
datetimestring=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
dump(rf, './data/interim/'+datetimestring+'_random_foest_base.joblib')

['./data/interim/11-09-2019_16-56-41_random_foest_base.joblib']

In [23]:
y_pred=rf.predict_proba(test)

In [32]:
from datetime import datetime as dt
import os
datetimestring=dt.now().strftime("%d-%m-%Y_%H-%M-%S")
os.mkdir('./data/predictions/'+datetimestring)
test['isFraud']=y_pred[:,1]
test['TransactionID']=list(test.index)
test[['TransactionID','isFraud']].to_csv('./data/predictions/'+datetimestring+'/prediction_results.csv',index = False)
dump(rf, './data/predictions/'+datetimestring+'/random_foest_base.joblib')

['./data/predictions/11-09-2019_17-03-09/random_foest_base.joblib']

In [15]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
rf = RandomForestClassifier(random_state = 42)
param = {'n_estimators': [300, 500],
        'max_depth': [50]}

gs = GridSearchCV(rf, param, cv=3, n_jobs=-1,scoring='roc_auc',verbose=2)
gs_fit = gs.fit(X_train, y_train)

temp = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 29.6min remaining: 29.6min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 43.1min finished


KeyboardInterrupt: 

In [16]:
temp

NameError: name 'temp' is not defined

In [42]:
results = pd.concat([temp,results]).sort_values('mean_test_score', ascending=False)

In [43]:
results[['params','mean_fit_time','mean_test_score']].sort_values('mean_test_score', ascending=False)

,params,mean_fit_time,mean_test_score
1,"{'max_depth': 100, 'n_estimators': 150}",700.032653,0.921341
3,"{'max_depth': 200, 'n_estimators': 150}",682.107487,0.921341
5,"{'max_depth': 500, 'n_estimators': 150}",467.296228,0.921341
0,"{'max_depth': 100, 'n_estimators': 100}",458.551123,0.918427
2,"{'max_depth': 200, 'n_estimators': 100}",470.277299,0.918427
4,"{'max_depth': 500, 'n_estimators': 100}",459.058936,0.918427
0,"{'max_depth': 750, 'n_estimators': 50}",231.455015,0.910674
1,"{'max_depth': 1000, 'n_estimators': 50}",229.743497,0.910674
2,"{'max_depth': 1200, 'n_estimators': 50}",176.848032,0.910674
1,"{'max_depth': 100, 'n_estimators': 50}",249.382044,0.910674


In [ ]:
dir(pd)

In [50]:
important_features

,0
C1,0.036231
C7,0.030252
C13,0.018790
C12,0.016223
TransactionDT,0.015808
card2,0.015740
TransactionAmt,0.014843
C14,0.014584
card1,0.014000
V265,0.013999


Accuracy: 0.9777576455447556
ROC area under: 0.9096866596536175


In [37]:
from FraudKagglePreProcessData import PreProcessData
from datetime import datetime
def process_data(transaction_path,identity_path,name,isTrain,model_feature_path=None):
    start=datetime.now()
    print(str(datetime.now())+' load_and_merge_data...')
    df,labels  = PreProcessData.load_and_merge_data(transaction_path,identity_path,isTrain=isTrain)
    print('df shape: ' + str(df.shape))
    
    print(str(datetime.now())+' process_dates...')
    df = PreProcessData.process_dates(df)
    
    print(str(datetime.now())+' process_device_names...')
    df = PreProcessData.process_device_names(df)
    
    print(str(datetime.now())+' get_lists_of_numerical_categorical...')
    numerical,categorical = PreProcessData.get_lists_of_numerical_categorical(df,'ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38')
    
    print(str(datetime.now())+' convert_numerical_categorical_to_strings...')
    df = PreProcessData.convert_numerical_categorical_to_strings(df,'ProductCD|card[1-6]|addr\d|\w_emaildomain|M[1-9]|time_|Device\w+|id_12|id_13|id_14|id_15|id_16|id_17|id_18|id_19|id_20|id_21|id_22|id_23|id_24|id_25|id_26|id_27|id_28|id_29|id_30|id_31|id_32|id_33|id_34|id_35|id_36|id_37|id_38')
    
    print(str(datetime.now())+' impute_missing_values...')
    df = PreProcessData.impute_missing_values(df,numerical,categorical)
    
    print(str(datetime.now())+' reduce_columns_with_PCA...')
    df,numerical,categorical = PreProcessData.reduce_columns_with_PCA(df,'^V.*',30,numerical,categorical)
    
    print(str(datetime.now())+' assign_low_freq_values_as_other_in_df...')
    df = PreProcessData.assign_low_freq_values_as_other_in_df(df,50,categorical)
    print('df shape: ' + str(df.shape))
    
    print(str(datetime.now())+' scale_numerical_fields...')
    df = PreProcessData.scale_numerical_fields(df,numerical)
    
    print(str(datetime.now())+' one_hot_encode_and_merge_with_numerical...')
    df = PreProcessData.one_hot_encode_and_merge_with_numerical(df,numerical,categorical,labels,isTrain)
    print('df shape: ' + str(df.shape))
    
    print(str(datetime.now())+' reduce_mem_usage...')
    df,NAs = PreProcessData.reduce_mem_usage(df)
          
    if isTrain == False:
        print(str(datetime.now())+' reconcile_features_from_test_to_train...')
        df = PreProcessData.reconcile_features_from_test_to_train(df,model_feature_path)
    
    print(str(datetime.now())+' pickle_df_and_columns...')
    PreProcessData.pickle_df_and_columns(df,name,isTrain)
    finish=datetime.now()
    print('time taken: '+str(finish-start))
    return df
#train = process_data('./data/raw/train_transaction_med.csv','./data/raw/train_identity_med.csv','train_med_processed',isTrain=True)
#validate = process_data('./data/raw/test_transaction_med.csv','./data/raw/test_identity_med.csv','test_med_processed',isTrain=False,model_feature_path='./data/processed/train_med_processed_feature_names.pkl')

#train = process_data('./data/raw/train_transaction.csv','./data/raw/train_identity.csv','train__processed',isTrain=True)
validate = process_data('./data/raw/test_transaction.csv','./data/raw/test_identity.csv','test_processed',isTrain=False,model_feature_path='./data/processed/train_processed_feature_names.pkl')

2019-09-05 13:29:29.878206 load_and_merge_data...
df shape: (506691, 432)
2019-09-05 13:29:54.997928 process_dates...
2019-09-05 13:29:59.549813 process_device_names...
2019-09-05 13:30:07.063725 get_lists_of_numerical_categorical...
2019-09-05 13:30:07.064722 convert_numerical_categorical_to_strings...
2019-09-05 13:30:17.540899 impute_missing_values...


C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


2019-09-05 13:30:49.385464 reduce_columns_with_PCA...
2019-09-05 13:31:04.011893 assign_low_freq_values_as_other_in_df...
df shape: (506691, 125)
2019-09-05 13:34:59.123403 scale_numerical_fields...
2019-09-05 13:35:05.619743 one_hot_encode_and_merge_with_numerical...
df shape: (506691, 1182)
2019-09-05 13:35:11.403272 reduce_mem_usage...
Memory usage of properties dataframe is : 841.9541463851929  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  700.8544340133667  MB
This is  83.24140180583264 % of the initial size
2019-09-05 13:35:46.299778 reconcile_features_from_test_to_train...
2019-09-05 13:36:31.761861 pickle_df_and_columns...
time taken: 0:07:06.908495


Create base model on train data

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, train.columns != 'isFraud'], train['isFraud'], test_size=0.2,random_state=42)
rf = RandomForestClassifier(random_state=42,n_estimators = 10, max_depth=25)
rf.fit(X_train, y_train)

KeyError: 'isFraud'

Evaluate model with test data

In [ ]:
y_pred_prob=rf.predict_proba(X_test)
y_pred=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("ROC area under:",metrics.roc_auc_score(y_test, y_pred_prob[:,1]))

Predict on validate data with model

In [35]:
y_pred=rf.predict_proba(validate)

ValueError: Number of features of the model must match the input. Model n_features is 842 and input n_features is 531 

Grid search CV on train data 

In [16]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
rf = RandomForestClassifier(random_state = 42)
param = {'n_estimators': [10,500,1000],
        'max_depth': [25,50]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1,scoring='roc_auc')
gs_fit = gs.fit(X_train, y_train)
pd.DataFrame(gs_fit.cv_results_)[['params','mean_fit_time','mean_test_score']].sort_values('mean_test_score', ascending=False)
#grid_results=[]
#grid_results = pd.concat([grid_results,pd.DataFrame(gs_fit.cv_results_)]).sort_values('mean_test_score', ascending=False)
#grid_results = pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)
#grid_results.to_pickle("./grid_search_results.pkl")
#grid_results[['params','mean_fit_time','mean_test_score']]

,params,mean_fit_time,mean_test_score
1,"{'max_depth': 25, 'n_estimators': 500}",1.195341,0.958123
4,"{'max_depth': 50, 'n_estimators': 500}",1.144995,0.958123
2,"{'max_depth': 25, 'n_estimators': 1000}",2.388538,0.953047
5,"{'max_depth': 50, 'n_estimators': 1000}",1.940876,0.953047
0,"{'max_depth': 25, 'n_estimators': 10}",0.035347,0.791869
3,"{'max_depth': 50, 'n_estimators': 10}",0.038656,0.791869


In [7]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                         train: 426.9MiB
                        labels:   9.0MiB
                            __:  22.8KiB
                            _2:  22.8KiB
                             _:  16.9KiB
                            _4:  16.9KiB
                           NAs:   3.2KiB
                PreProcessData:   1.0KiB
                           _i7:   578.0B
                      datetime:   400.0B
